In [ ]:
# 📚 Multi-label Dog Breed Classifier with ResNet50
# By: ChatGPT + 이상현님

In [ ]:
# ✅ 1. Import Libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
# ✅ 2. Multi-label Dataset Class
class MultiLabelDogDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.class_names = list(self.annotations.columns[1:])
        self.num_classes = len(self.class_names)

In [ ]:
def __len__(self):
        return len(self.annotations)

In [ ]:
def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        labels = torch.tensor(self.annotations.iloc[idx, 1:].values.astype(np.float32))
        if self.transform:
            image = self.transform(image)
        return image, labels

In [ ]:
# ✅ 3. Settings
data_dir = "./dog_training_set"
csv_path = "./labels.csv"
batch_size = 32
num_epochs = 10

In [ ]:
# ✅ 4. Image Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
# ✅ 5. Dataset & DataLoader
train_dataset = MultiLabelDogDataset(csv_file=csv_path, root_dir=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
class_names = train_dataset.class_names
num_classes = train_dataset.num_classes

In [ ]:
# ✅ 6. Model Definition
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# ✅ 7. Loss & Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# ✅ 8. Training Loop
print("🚀 Training Started...")
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i + 1) % 1 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss:.4f}")
            running_loss = 0.0

In [ ]:
# ✅ 9. Save Model
model_path = "./resnet50_multilabel.pth"
torch.save(model.state_dict(), model_path)
print(f"✅ Model saved at: {model_path}")

In [ ]:
# ✅ 10. Prediction Function
def load_model(model_path, num_classes):
    model = models.resnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    model.eval()
    return model

In [ ]:
def predict_sample(image_path, model, class_names, threshold=0.3):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image_tensor)
        probs = torch.sigmoid(output[0])
    predicted_indices = (probs > threshold).nonzero(as_tuple=True)[0]
    predicted_labels = [class_names[i] for i in predicted_indices]
    confidences = [probs[i].item() for i in predicted_indices]
    plt.imshow(image)
    plt.title("Predicted: " + ', '.join([f"{l} ({c:.2f})" for l, c in zip(predicted_labels, confidences)]))
    plt.axis('off')
    plt.show()
    print("🔍 Predicted Breeds:")
    for label, conf in zip(predicted_labels, confidences):
        print(f"→ {label}: {conf:.2f}")

In [ ]:
# ✅ 11. Run prediction (예시)
# sample_image = "./dog_training_set/Labradoodle/your_sample.jpg"
# model_loaded = load_model(model_path, num_classes)
# predict_sample(sample_image, model_loaded, class_names)